# Camera Geometries

See https://cta-observatory.github.io/ctapipe/instrument/index.html
(or https://github.com/cta-observatory/ctapipe/blob/70a023d9c3d6ba4eada031dac59b1d8c152fc4b0/docs/instrument/index.rst)

In [ ]:
%matplotlib inline

import matplotlib
matplotlib.rcParams['figure.figsize'] = (6, 6)

import math
import numpy as np
#np.set_printoptions(threshold=np.nan)

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

from astropy import units as u

import ctapipe
from ctapipe.io.hessio import hessio_event_source
import pyhessio

# Old version
from ctapipe.io import camera

# New version
#from ctapipe.instrument import camera

In [ ]:
ctapipe.__file__

In [ ]:
ctapipe.__version__

## Construct a CameraGeometry

### Method #1: make from scratch

In [ ]:
# camera identification string
cam_id = "foo"

In [ ]:
# array(int): pixels id numbers
pix_id = np.arange(1, 7)
pix_id

In [ ]:
# array with units: position of each pixel (x-coordinate)
pix_x = np.array([1., 2., 3., 1., 2., 3.]) / 100. * u.meter
pix_x

In [ ]:
# array with units: position of each pixel (y-coordinate)
pix_y = np.array([1., 1., 1., 2., 2., 2.]) / 100. * u.meter
pix_y

In [ ]:
# array(float) surface area of each pixel
pix_area = np.ones(6) / 100. * (u.meter ** 2)
pix_area

In [ ]:
# list(arrays): adjacency list for each pixel
neighbors = np.array([[2, 4], [1, 3, 5], [2, 6], [1, 5], [4, 2, 6], [3, 5]])

In [ ]:
# either 'rectangular' or 'hexagonal'
pix_type = 'rectangular'

In [ ]:
geom = camera.CameraGeometry(cam_id, pix_id, pix_x, pix_y, pix_area, neighbors, pix_type)
geom.to_table()

### Method #2: *CameraGeometry.from_name()*

Fails for astri:

> No such file or directory: '.../ctapipe-extra/datasets/astri_camgeom.fits.gz'

In [ ]:
#geom = camera.CameraGeometry.from_name("astri", 1)
#geom = camera.CameraGeometry.from_name("HESS", 1)

### Method #3: "guess()"

Load a Monte-Carlo file, get the list of pixel X and Y positions and the telescope focal length and use ``CameraGeometry.guess(x,y,flen)`` - this will work for all telescopes in CTA so far.

In [ ]:
tel_id = 1
source = hessio_event_source("/Users/jdecock/data/astri_mini_array/proton/run10001.simtel.gz", allowed_tels=[tel_id])

In [ ]:
for ev in source:
    event = ev

In [ ]:
pix_x = event.inst.pixel_pos[tel_id][0]
pix_y = event.inst.pixel_pos[tel_id][1]
optical_foclen = event.inst.optical_foclen[tel_id]

geom = camera.CameraGeometry.guess(pix_x, pix_y, optical_foclen)
geom.to_table()

In [ ]:
print("cam_id:", geom.cam_id)
print("cam_rotation:", geom.cam_rotation)
#print("neighbors:", geom.neighbors)
print("pix_area:", geom.pix_area)
print("pix_id:", geom.pix_id)
print("pix_rotation:", geom.pix_rotation)
print("pix_type:", geom.pix_type)
print("pix_x:", geom.pix_x)
print("pix_y:", geom.pix_y)

### Method #4: load from table (fits file, hdf5 file, ...)

Load from file (table) https://cta-observatory.github.io/ctapipe/instrument/index.html#input-output

Doesn't work on ctapipe 0.4.0 (too old version ?)

In [ ]:
geom.to_table().write('astri.fits.gz')

In [ ]:
!ls 'astri.fits.gz'

In [ ]:
#geom = camera.CameraGeometry.from_table('astri.fits.gz')

### Method #5: make a rectangular camera

In [ ]:
num_pixels_x, num_pixels_y, range_x, range_y = 5, 3, (-2., 2.), (-1., 1.)
geom = ctapipe.io.camera.make_rectangular_camera_geometry(num_pixels_x, num_pixels_y, range_x, range_y)

In [ ]:
print("cam_id:", geom.cam_id)
print("cam_rotation:", geom.cam_rotation)
#print("neighbors:", geom.neighbors)
print("pix_area:", geom.pix_area)
print("pix_id:", geom.pix_id)
print("pix_rotation:", geom.pix_rotation)
print("pix_type:", geom.pix_type)
print("pix_x:", geom.pix_x)
print("pix_y:", geom.pix_y)

### Method #6: *CameraGeometry.from_file()*

Doesn't work.

In [ ]:
#import pyhessio as hessio

In [ ]:
#geom = camera.CameraGeometry.from_file("/Users/jdecock/data/astri_mini_array/proton/run10001.simtel.gz", 1)

### My method: JSON

#### Make the geom.json file

In [ ]:
tel_id = 1
source = hessio_event_source("/Users/jdecock/data/astri_mini_array/proton/run10001.simtel.gz", allowed_tels=[tel_id])

In [ ]:
for ev in source:
    event = ev

In [ ]:
pix_x = event.inst.pixel_pos[tel_id][0]
pix_y = event.inst.pixel_pos[tel_id][1]
optical_foclen = event.inst.optical_foclen[tel_id]

geom = camera.CameraGeometry.guess(pix_x, pix_y, optical_foclen)
geom.to_table()

In [ ]:
print("cam_id:", geom.cam_id)
print("cam_rotation:", geom.cam_rotation)
#print("neighbors:", geom.neighbors)
print("pix_area:", geom.pix_area)
print("pix_id:", geom.pix_id)
print("pix_rotation:", geom.pix_rotation)
print("pix_type:", geom.pix_type)
print("pix_x:", geom.pix_x)
print("pix_y:", geom.pix_y)

In [ ]:
json_dict = {"cam_id": geom.cam_id,
             "cam_rotation": geom.cam_rotation.value,
             "neighbors": geom.neighbors,
             "pix_area": geom.pix_area.value.tolist(),
             "pix_id": geom.pix_id.tolist(),
             #"pix_rotation": geom.pix_rotation,
             "pix_type": geom.pix_type,
             "pix_x": geom.pix_x.value.tolist(),
             "pix_y": geom.pix_y.value.tolist()}

#json_dict

In [ ]:
import json

In [ ]:
json_file_path = "astri.geom.json"

with open(json_file_path, "w") as fd:
    #json.dump(data, fd)                           # no pretty print
    json.dump(json_dict, fd, sort_keys=True, indent=4)  # pretty print format

#### Make the geom object from the geom.json file

In [ ]:
with open(json_file_path, 'r') as fd:
    json_dict = json.load(fd)

cam_id = json_dict['cam_id']
pix_id = np.array(json_dict['pix_id'])
pix_x =  np.array(json_dict['pix_x']) * u.meter
pix_y =  np.array(json_dict['pix_y']) * u.meter
pix_area =  np.array(json_dict['pix_area']) * (u.meter ** 2)
neighbors = json_dict['neighbors']
pix_type =  json_dict['pix_type']

geom = camera.CameraGeometry(cam_id, pix_id, pix_x, pix_y, pix_area, neighbors, pix_type)

print("cam_id:", geom.cam_id)
print("cam_rotation:", geom.cam_rotation)
#print("neighbors:", geom.neighbors)
print("pix_area:", geom.pix_area)
print("pix_id:", geom.pix_id)
print("pix_rotation:", geom.pix_rotation)
print("pix_type:", geom.pix_type)
print("pix_x:", geom.pix_x)
print("pix_y:", geom.pix_y)